In [21]:
# !python -m pip install mlagents==0.28.0
# !python -m pip install gym
# !cd "/Users/aditya/Documents/GitHub/game_creation_research/ml-agents/gym-unity" && pip3 install -e .

import mlagents
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from gym_unity.envs import UnityToGymWrapper

In [24]:
channel = EngineConfigurationChannel()
channel.set_configuration_parameters(time_scale = 1.0, quality_level=1, target_frame_rate=5)
env = UnityEnvironment(file_name="/Users/aditya/Documents/GitHub/game_creation_research/Object Physics Sandbox/MLAgentView_20April22.app", seed=1, side_channels=[channel], worker_id=2)

# Start interacting with the environment.
env.reset()
env = UnityToGymWrapper(env, allow_multiple_obs=True)

E0420 11:45:45.758191000 4753563136 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers


[INFO] Connected to Unity environment with package version 2.2.1-exp.1 and communication version 1.5.0
[INFO] Connected new brain: PlaceAndShoot?team=0


In [25]:
env.step(action=[0.1, 0.5, 0, 0, 0, 0])

([array([ 5.34      , -2.65      , -2.8114905 ,  0.10777473,  5.34      ,
         -1.25      ,  5.34      ,  0.24      ,  5.34      ,  3.26      ,
          1.        , -4.1317406 ,  0.        ], dtype=float32)],
 0.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7fbff21d9670>})

In [26]:
env.step(action=[0.1, 0.5, 0.1, 0.1, 1, 0])

([array([ 5.34      , -2.65      , -2.8114905 ,  0.10777473,  5.34      ,
         -1.25      ,  5.34      ,  0.24      ,  1.        ,  1.        ,
          1.        , -2.9022753 ,  0.        ], dtype=float32)],
 0.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7fbff2212d90>})

In [27]:
env.close()